In [ ]:

# Purpose: benchmark Evro Line’s operational & financial KPIs against peers, analyze trends, and generate data-driven pricing/cost insights

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from typing import List

# File and basic configuration
EXCEL_PATH   = "kpi_data.xlsx"          # path to Excel workbook (all companies, all periods)
DATE_COL     = "date"                   # column holding period end-date
COMPANY_COL  = "company"                # column with legal entity / brand name
OUR_FIRM     = "Evro Line"              # exact string as it appears in the data

# Load source data: one row per company per period (quarter or month)
df_raw = pd.read_excel(EXCEL_PATH, parse_dates=[DATE_COL])

# Add KPIs: unit economics, financial ratios, trend indicators
def add_all_kpis(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    safe_div = lambda num, den: np.where(den != 0, num / den, np.nan)

    # Operational KPIs – per shipment (invoice)
    df["rev_per_op"]   = safe_div(df["revenue"], df["num_invoices"])            # price level: average revenue per shipment
    df["cost_per_op"]  = safe_div(df["cogs"],    df["num_invoices"])            # direct cost per shipment (fuel, carriers, etc.)
    df["gp_per_op"]    = safe_div(df["revenue"] - df["cogs"], df["num_invoices"])  # gross profit per shipment
    df["ebit_per_op"]  = safe_div(df["ebit"],    df["num_invoices"])            # operating profit per shipment
    df["opex_per_op"]  = safe_div(df["opex"],    df["num_invoices"])            # overhead allocation per shipment
    df["capex_per_op"] = safe_div(df["capex"],   df["num_invoices"])            # investment (capex) apportioned per shipment

    # Financial ratios (standard)
    df["gross_margin"]   = safe_div(df["revenue"] - df["cogs"], df["revenue"])  # gross margin %
    df["ebit_margin"]    = safe_div(df["ebit"],                df["revenue"])   # EBIT margin %
    df["net_margin"]     = safe_div(df["net_income"],          df["revenue"])   # net profit margin %
    df["opex_ratio"]     = safe_div(df["opex"],                df["revenue"])   # SG&A as % of revenue
    df["capex_ratio"]    = safe_div(df["capex"],               df["revenue"])   # capex as % of revenue
    df["asset_turnover"] = safe_div(df["revenue"],             df["total_assets"])  # revenue generated per asset €
    df["debt_to_equity"] = safe_div(df["liabilities"],         df["equity"])        # leverage ratio

    # Trend analysis – period-over-period % change
    trend_targets = [
        "revenue", "cogs", "opex", "ebit", "net_income",
        "gross_margin", "ebit_margin", "net_margin", "asset_turnover"
    ]
    df.sort_values([COMPANY_COL, DATE_COL], inplace=True)
    for col in trend_targets:
        df[f"{col}_growth_pct"] = (
            df.groupby(COMPANY_COL)[col]
              .pct_change()
              .replace([np.inf, -np.inf], np.nan)
              .round(4)
        )

    return df

df_kpi = add_all_kpis(df_raw)

# Build peer-variance table: Evro vs average of competitors
def build_peer_variance(df_kpi: pd.DataFrame, kpi_cols: List[str]) -> pd.DataFrame:
    ours = (
        df_kpi[df_kpi[COMPANY_COL] == OUR_FIRM]
        .set_index(DATE_COL)[kpi_cols]
        .add_suffix("_ours")
    )
    peers = (
        df_kpi[df_kpi[COMPANY_COL] != OUR_FIRM]
        .groupby(DATE_COL)[kpi_cols]
        .mean()
        .add_suffix("_peer_avg")
    )
    variance = ours.join(peers)
    for kpi in kpi_cols:
        variance[f"{kpi}_var"] = variance[f"{kpi}_ours"] - variance[f"{kpi}_peer_avg"]
    return variance

peer_kpis = [
    "rev_per_op", "cost_per_op", "gp_per_op",
    "revenue_growth_pct", "gross_margin", "ebit_margin", "net_margin"
]

peer_var = build_peer_variance(df_kpi, peer_kpis)

# Plot: Evro Line (solid) vs peer average (dashed) for a selected KPI
def dual_line_plot(df_var: pd.DataFrame, base_col: str):
    fig, ax = plt.subplots(figsize=(10, 4))
    ax.plot(df_var.index, df_var[f"{base_col}_ours"], label="Evro Line")
    ax.plot(df_var.index, df_var[f"{base_col}_peer_avg"], linestyle="--", label="Peer Avg")
    ax.set_title(f"{base_col.replace('_',' ').title()} – Evro vs Peers")
    ax.xaxis.set_major_formatter(DateFormatter("%Y-%m"))
    ax.grid(alpha=0.3); ax.legend(); fig.tight_layout()
    return fig

dual_line_plot(peer_var, "ebit_margin").savefig("ebit_margin_evro_vs_peers.png")
peer_var.to_excel("peer_variance_output.xlsx")

# STRATEGIC INTERPRETATION (Q1–Q4)
latest = peer_var.iloc[-1][[
    "rev_per_op_var", "cost_per_op_var", "gp_per_op_var", "ebit_margin_var"
]].round(2)

print("\n--- STRATEGIC BENCHMARK SUMMARY (most recent period) ---")
print(latest)

print("\n--- INTERPRETATION ---")
# Q1 – Pricing gap: Do we charge more or less than peers?
if latest["rev_per_op_var"] > 0:
    print("Pricing: Evro Line charges more per shipment than the peer average.")
else:
    print("Pricing: Evro Line charges less per shipment than peers.")

# Q2 – Direct cost gap: Are our unit costs higher or lower?
if latest["cost_per_op_var"] > 0:
    print("Costs: Direct cost per shipment is higher than peer average.")
else:
    print("Costs: Direct cost per shipment is lower than peer average.")

# Q3 – Unit margin gap: Are we keeping enough gross profit per job?
if latest["gp_per_op_var"] < 0:
    print("Margins: Gross profit per shipment is below peer average.")
else:
    print("Margins: Gross profit per shipment is above peer average.")

# Q4 – Overall profitability gap: Is EBIT margin competitive?
if latest["ebit_margin_var"] < 0:
    print("Profitability: EBIT margin is below peer average.")
else:
    print("Profitability: EBIT margin is above peer average.")

print("\n--- RECOMMENDED NEXT STEP ---")
# Decision rule 1 – Premium pricing but weak EBIT → efficiency focus
if (latest["rev_per_op_var"] > 0) and (latest["ebit_margin_var"] < 0):
    print("Evro charges a premium but retains less profit; investigate overhead, process efficiency, and SG&A.")
# Decision rule 2 – Low price but healthy gross profit → consider raising price
elif (latest["rev_per_op_var"] < 0) and (latest["gp_per_op_var"] > 0):
    print("Evro operates efficiently yet undercharges; evaluate targeted price increases or premium positioning.")
# Decision rule 3 – No major gaps
else:
    print("No critical pricing-cost misalignment detected; continue quarterly monitoring and peer benchmarking.")
